---
title: |
  | Earnings Management and Investor Protection:
  | Accounting Reading Group - Assignment III\vspace{1cm}
author:
  - name: Melisa Mazaeva
    email: melisa.mazaeva@student.hu-berlin.de
    affiliations:
      - Humboldt-Universität zu Berlin  
date: today
date-format: MMM D, YYYY [\vspace{1cm}]
abstract: |
  | This project uses the TRR 266 Template for Reproducible Empirical Accounting Research (TREAT) to provide an infrastructure for open science-oriented empirical projects. Leveraging external Worldscope data sets on financial data, the repository showcases a reproducible workflow that integrates Python scripts for data analysis. The project’s output demonstrates a comprehensive application of skills to replicate and extend the findings from the seminal paper by Leuz, Nanda, and Wysocki (2003), particularly in providing descriptive statistics for the four individual earnings management measures as well as the aggregate earnings management score across various countries. In doing so, it documents and discusses the research design choices made and the variations between the original and reproduced results. This code base, adapted from TREAT, should give you an overview on how the template is supposed to be used for my specific project and how to structure a reproducible empirical project.
  | \vspace{6cm}
bibliography: references.bib
biblio-style: apsr
format:
  pdf:
    documentclass: article
    number-sections: true
    toc: false
fig_caption: yes
fontsize: 11pt
ident: yes
always_allow_html: yes
header-includes:
  - \usepackage[nolists]{endfloat}    
  - \usepackage{setspace}\doublespacing
  - \setlength{\parindent}{4em}
  - \setlength{\parskip}{0em}
  - \usepackage[hang,flushmargin]{footmisc}
  - \usepackage{caption} 
  - \captionsetup[table]{skip=24pt,font=bf}
  - \usepackage{array}
  - \usepackage{threeparttable}
  - \usepackage{adjustbox}
  - \usepackage{graphicx}
  - \usepackage{csquotes}
  - \usepackage{indentfirst}  # Added this line to ensure the first paragraph is indented for better readability
  - \usepackage[margin=1in]{geometry}
---


\pagebreak

## delete or take fikirs below

In [ ]:
#| echo: false
#| output: false

import pickle

with open('../output/results.pickle', 'rb') as f:
        results = pickle.load(f)

min_fyear, max_fyear, no_unique_firms = results['Desc information'].values()


def rename_var_to_label(table, italic=True):
    for var_name, var_label in results['Variable Names'].items():
        var_label = f'\\textit{{{var_label}}}' if italic else var_label
        table = table.replace(var_name, var_label)
    return table

## from fikir

In [ ]:
#| echo: false
#| output: false

import pickle
import pandas as pd

with open('../output/results.pickle', 'rb') as f:
        results = pickle.load(f)

def escape_underscores(df):
    df.columns = [col.replace('_', '\_') for col in df.columns]
    return df

# Custom formatter function
def custom_float_format(x):
    return f"{x:.3f}".lstrip('0') if pd.notnull(x) else ""


def prep_latex_table(df, caption=None, label=None):
    df = escape_underscores(df.reset_index())
    num_columns = len(df.columns)
    column_format = 'l' + 'r' * (num_columns - 1)
    latex_table = df.to_latex(
        column_format=column_format,
        index=False,
        float_format=custom_float_format,
    )
    latex_table_lines = [
        "\\begin{table}[htbp]",
        "\\centering",
        "\\resizebox{\\textwidth}{!}{%",
        latex_table,
        "}",
        f"\\caption{{{caption}}}" if caption else "",
        f"\\label{{{label}}}" if label else "",
        "\\end{table}"
    ]
    return "\n".join(line for line in latex_table_lines if line)

\pagebreak


# Research Design Choices and Assumptions

The aim of Assignment III is to replicate a specific empirical table (Table 2 Panel A) from the seminal paper by @Leuz_2003. This table involves calculating the EM measures for firms across various countries over a defined period and examining the relationship between these measures and investor protection. The replication process includes data loading, preparation, cleaning, and normalization, followed by the application of statistical methods to compute and interpret financial metrics. For Assignment III, I pulled data from the Worldscope database through WRDS and used the Python programming language to carry out the empirical analysis. Visual Studio Code was used as the Integrated Development Environment (IDE) for writing, debugging, and optimizing the Python code.

The replication is based on data pulled from the Worldscope database, specifically from the `wrds_ws_company` and `wrds_ws_funda` tables, which were merged for the analysis. The first table provides company profile information, including items such as ISIN, Worldscope Identifiers, company name, and the country where the company is domiciled. The latter table contains Fundamentals Annual data at the company-year level, including items such total assets, net income, and other relevant financial variables.

Following @Leuz_2003, I focus the analysis on companies across various countries, ensuring that the data accurately reflects the fiscal years 1990 to 1999 as specified in the original study. The replication aims to mirror the research design as closely as possible with the available data.

In addition, I impose the following assumptions to ensure clarity and consistency where the paper does not provide explicit guidance:

## check assumptions in Fikir and mine
1. The original paper references the November 2000 version of the Worldscope Database. However, the data used for this analysis represents the latest available version, updated in July 2024, with quarterly frequency updates [@WRDS_2024]. Due to potential adjustments and updates made to the database since November 2000, there may be differences between the databases that could affect the results. For example, companies may restate financials after the original reporting period, so that these restatements are reflected in the later database version rather than the historical one. Moreover, the data vendor Refinitiv regularly updates its databases to correct errors and add new information, which may be included in the later data but not in the November 2000 snapshot.
2. While pulling the data for analysis, I encountered negative values for some key financial metrics such as operating income (`item1250`), net income before extraordinary items/preferred dividends (`item1551`), and net income before preferred dividends (`item1651`). The paper by @Leuz_2003 does not explicitly specify how to handle negative values in key financial metrics. For the purpose of this replication study, I will include negative values in the analysis. Including these values ensures that the analysis captures the full spectrum of earnings management activities across different countries.
3. The EM measures are based on scaled variables (e.g., operating cash flow scaled by lagged total assets). As such, the currency of the relevant data items should not affect the results as long as the same currency is used consistently for both the numerator and the denominator. This approach ensures comparability across different countries, regardless of their local currencies. Additionally, according to a document by [@Thomson_Financial_2007, p.20], all Worldscope data is consistently reported in the local currency of each firm’s country of domicile, eliminating the need for currency conversions in this project.
333. ## delete. mine 
333. It is assumed that Penman (2013) restricted the possible P/B values to a range of 0 to 7 to exclude outliers. Hence, extreme P/B values that are negative and very high (greater than 7) are excluded to focus the analysis on firms with more stable and reasonable valuations, reducing the impact of outliers.

By following the steps provided in Section 4 and adhering to the assumptions made, I successfully replicated the analysis and produced the required table. A thorough step-by-step approach, with each step clearly documented,  helped to understand and verify the outputs.


# Replication Steps
## Step 1: Pulling the Data and Managing the Databases
In contrast to Assignment I, where the data was provided externally, Assignment III involves additionally pulling data directly from the Worldscope database, merging relevant tables, and preparing the data for further analysis from raw data to final output.

To compile the dataset, the dynamic and static datasets were merged on the `item6105`  identifier, representing the unique Worldscope Permanent ID. WRDS advises using this identifier consistently within Worldscope data since it remains stable over time [@WRDS_Overview_2024].

\pagebreak

\setcounter{table}{0}
\renewcommand{\thetable}{\arabic{table}}

# References {-}
\setlength{\parindent}{-0.2in}
\setlength{\leftskip}{0.2in}
\setlength{\parskip}{8pt}
\noindent